In [1]:
"""
e_3_student_scoring.ipynb
───────────────────────────────────────────────────────────────────────────────
LOOCV student scoring with per-fold hyperparameters reused from teacher models
(no tuning) and percent-based augmentation sizes. Student embedding is selectable;
labels are read from a configurable teacher label source.

This script:
1) Loads seed data and resources
   - Reads 96 seeds (texts + 14 targets).
   - Loads or computes seed sentence embeddings for the selected student embedding.
   - Discovers augmentation methods and their M_max from tuned teacher outputs.
   - Preloads synthetic text embeddings for the selected student embedding and
     per-fold tuned labels from the label source (e.g., e5_base + chain_ERCcv_lr).

2) Builds student regressors from teacher hyperparameters (no tuning)
   - Students: chain_ERCcv_lr, local_lasso, local_rf, global_rf, chain_ERCcv_rf.
   - Extracts per-fold HPs from teacher pickles (step_g_2a) for the same
     student embedding; raises an error if missing (no silent defaults).

3) Evaluates the baseline (seeds-only)
   - For each LOOCV fold and student, loads that fold’s teacher estimator
     (if present) and predicts the held-out seed; otherwise rebuilds the
     same pipeline from the teacher’s stored HPs and predicts.
   - Writes per-pct baseline files for naming symmetry.

4) Evaluates the augmented “full” variant (seeds + synthetics)
   - Percent mode: P ∈ {10, 20, 50, 100, 200, 400}; K = round(P% of 96).
   - For each fold, takes the first K items from that fold’s M_max labeled set
     (labels from the label source), fits the student with the per-fold HPs,
     and predicts the held-out seed.
   - Idempotent: computes only missing folds and preserves already complete CSVs.
   - Legacy reuse for chain_ERCcv_lr at {100,200,400} applies only when the
     student embedding equals the label-source embedding (e.g., e5_base).

5) Produces unified summary tables (no PRIMARY/APPENDIX split)
   - Aggregates per-fold median RRMSE into a wide summary with columns:
     baseline, full, baseline_vs_full.
   - Writes a comparison table with ΔRRMSE and relative % change.

6) Performs statistical analyses
   - Per-config Wilcoxon signed-rank tests (one-sided, alternative="less",
     zero_method="pratt") with Holm correction grouped by (regressor, method).
   - Paired Cliff’s delta per configuration.
   - Pooled Wilcoxon and Cliff’s delta across students per (method, pct).
   - Hierarchical bootstrap (folds × domains micro-bootstrap) for Δ median with 95% CI.

7) Visualizes performance
   - RRMSE vs %K and ΔRRMSE vs %K for the configured PCTS_FOR_PLOTS.

Idempotency & disk reuse:
- Reuses seed embeddings if shape and max_seq_len match; otherwise rebuilds.
- Reuses synthetic embeddings (cache); builds them if missing.
- Skips FULL computations for (student, method, pct) whose per-fold CSV is complete.
- Optional: skip recomputing baseline if all baseline CSVs are already complete.
- Optional: summarization-only mode to build tables/plots from existing CSVs.

Inputs:
- data/activity_scores.csv
- data/activities.csv
- outputs/results/{student_embedding}_vectors.npy
- outputs/e_1_synth_augmentation/g_final_n{M}_{method}.csv
- outputs/e_2_teacher_labeling/g2f_labels_fold{ii}_n{M}_{method}__{label_embedding}__{label_model}.csv
- models/teacher/teacher_fold{ii}_{student_embedding}__{student}.pkl
- outputs/e_2_teacher_labeling/cache/synth_embeds/*__{student_embedding}.npy and *__index.csv
- (legacy reuse, only if student_embedding==label_embedding==e5_base and student==chain_ERCcv_lr)
  outputs/e_3_student_scoring/results/
    rrmse_perfold_e5_base__chain_ERCcv_lr__{method}__n96_sps1__full.csv
    rrmse_perfold_e5_base__chain_ERCcv_lr__{method}__n192_sps2__full.csv
    rrmse_perfold_e5_base__chain_ERCcv_lr__{method}__n384_sps4__full.csv

Outputs:
- outputs/e_3_student_scoring/results/
    rrmse_perfold_{student_embedding}__{regressor}__{method}__pct{P}_K{K}__Mmax{Mmax}__baseline.csv
    rrmse_perfold_{student_embedding}__{regressor}__{method}__pct{P}_K{K}__Mmax{Mmax}__full.csv
- outputs/e_3_student_scoring/hp_perfold/
    {regressor}/{method}/n{Mmax}/pct{P}_K{K}/fold{ii}_best.json
- outputs/e_3_student_scoring/tables/
    summary_median_rrmse.csv
    summary_median_rrmse_with_delta.csv
    wilcoxon_holm_vs_baseline.csv
    cliffs_delta_vs_baseline.csv
    wilcoxon_pooled_vs_baseline.csv
    cliffs_delta_pooled_vs_baseline.csv
    bootstrap_delta_ci.csv
- outputs/e_3_student_scoring/plots/
    combined_rrmse_vs_pct__{method}.png
    combined_delta_vs_pct__{method}.png
    combined_rrmse_vs_pct__{method}__{embedding|all}.png
    combined_delta_vs_pct__{method}__{embedding|all}.png
- outputs/e_3_student_scoring/
    cache/, run.log, run_config.json
"""

# ────────────────────────────────────────────────────────────────────────────
#  Imports
# ────────────────────────────────────────────────────────────────────────────

from __future__ import annotations

from itertools import combinations, cycle
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import json
import logging
import os
import pickle
import re
import sys
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scikit_posthocs as sp
from scipy.stats import friedmanchisquare, wilcoxon
from sentence_transformers import SentenceTransformer
from statsmodels.stats.multitest import multipletests
import torch
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state

warnings.filterwarnings("ignore")
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# ────────────────────────────────────────────────────────────────────────────
#  Paths 
# ────────────────────────────────────────────────────────────────────────────
def project_root(marker: str = "LICENSE") -> Path:
    here = Path.cwd().resolve()
    for d in (here, *here.parents):
        if (d / marker).is_file():
            return d
    return Path.cwd().resolve()

ROOT = project_root()
DATA_DIR = ROOT / "data"

G1_DIR  = ROOT / "outputs" / "e_1_synth_augmentation"
G2_DIR  = ROOT / "outputs" / "e_2_teacher_labeling"
G3A_DIR = ROOT / "outputs" / "e_3_student_scoring"

RES_DIR     = G3A_DIR / "results"
TABLES_DIR  = G3A_DIR / "tables"
CACHE_DIR   = G3A_DIR / "cache"
HP_PERFOLD  = G3A_DIR / "hp_perfold"
FIGS_DIR    = G3A_DIR / "plots"

for p in (G3A_DIR, RES_DIR, TABLES_DIR, CACHE_DIR, HP_PERFOLD, FIGS_DIR):
    p.mkdir(parents=True, exist_ok=True)

OUT_TABLES = TABLES_DIR
PLOTS_DIR = FIGS_DIR
OUT_PLOTS  = FIGS_DIR

LOG_FILE = G3A_DIR / "run.log"
for h in list(logging.root.handlers):
    logging.root.removeHandler(h)

# ────────────────────────────────────────────────────────────────────────────
#  Logging 
# ────────────────────────────────────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.FileHandler(str(LOG_FILE), mode="a", encoding="utf-8"),
        # use the uncaptured stdout to bypass notebook/pytest capturing
        logging.StreamHandler(getattr(sys, "__stdout__", sys.stdout)),
    ],
    force=True,
)
log = logging.getLogger(__name__)
_say_logger = logging.getLogger("sayfile")
if not _say_logger.handlers:
    _say_logger.setLevel(logging.INFO)
    _say_logger.propagate = False  # do NOT bubble to root (prevents console echo)
    _fh = logging.FileHandler(str(LOG_FILE), mode="a", encoding="utf-8")
    _fh.setFormatter(logging.Formatter("%(asctime)s %(levelname)s: %(message)s",
                                       datefmt="%Y-%m-%d %H:%M:%S"))
    _say_logger.addHandler(_fh)
    
RUN_ID: str = time.strftime("%Y%m%d-%H%M%S")
# ────────────────────────────────────────────────────────────────────────────
#  Config
# ────────────────────────────────────────────────────────────────────────────
REVIEW_MODE: bool = True

N_TARGETS = 14

METHODS: Optional[List[str]] = None

PCT_LIST: List[int]       = [10, 20, 50, 100, 200, 400]
PCTS_FOR_PLOTS: List[int] = [10, 20, 50, 100, 200, 400]
PRIMARY_PCTS = PCT_LIST

SAVE_PRED_PERFOLD: bool = True
LOG_EVERY: int          = 16
MAX_FOLDS: int          = 0  # 0 ⇒ all 96

# Which student regressors to run
STUDENTS: List[str] = ["chain_ERCcv_lr", "local_lasso", "local_rf", "global_rf", "chain_ERCcv_rf"]

TARGET_COLS = [f"domain{i}" for i in range(1, 15)]
_DOM_PREFIX = "rrmse_domain"
BOOT_B: int = 5000

# ────────────────────────────────────────────────────────────────────────────
#  Device, cores, seeding
# ────────────────────────────────────────────────────────────────────────────
DETERMINISTIC = True
N_JOBS: int = min(os.cpu_count() or 6, 6)
os.environ.setdefault("OMP_NUM_THREADS",        str(N_JOBS))
os.environ.setdefault("OPENBLAS_NUM_THREADS",   str(N_JOBS))
os.environ.setdefault("MKL_NUM_THREADS",        str(N_JOBS))
os.environ.setdefault("VECLIB_MAXIMUM_THREADS", str(N_JOBS))
os.environ.setdefault("NUMEXPR_NUM_THREADS",    str(N_JOBS))
try:
    torch.set_num_threads(N_JOBS)
except Exception:
    pass

device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available()
    else "cpu"
)
DEVICE_STR = device.type

SEED = 42
np.random.seed(SEED)
try:
    import random
    random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        torch.manual_seed(SEED)
    if DETERMINISTIC:
        torch.use_deterministic_algorithms(True)
except Exception:
    pass

# ────────────────────────────────────────────────────────────────────────────
#  Embedding registry and selection
# ────────────────────────────────────────────────────────────────────────────
EMBEDDING_SPECS: Dict[str, str] = {
    "e5_base":          "embaas/sentence-transformers-multilingual-e5-base",
    #"e5_large":         "embaas/sentence-transformers-multilingual-e5-large",
    #"simcse_xlmr_base": "sentence-transformers/paraphrase-xlm-r-multilingual-v1",
    #"sbert_bert":       "jegormeister/bert-base-dutch-cased-snli",
}

STUDENT_EMBEDDINGS: List[str] = ["e5_base"]
def _normalize_embeddings(x):
    if isinstance(x, str):
        return [s.strip() for s in x.split(",") if s.strip()]
    elif isinstance(x, (list, tuple)):
        return [str(s).strip() for s in x if str(s).strip()]
    else:
        return []

# Normalize & dedupe
_seen = set()
STUDENT_EMBEDDINGS = [e for e in STUDENT_EMBEDDINGS if not (e in _seen or _seen.add(e))]
# After deduping STUDENT_EMBEDDINGS
STUDENT_EMB = STUDENT_EMBEDDINGS[0]

# Label source (teacher that produced the labels for synthetics)
LABEL_EMB: str   = "e5_base"
LABEL_MODEL: str = "chain_ERCcv_lr"
if LABEL_EMB not in EMBEDDING_SPECS:
    raise ValueError(f"Unknown LABEL_SOURCE_EMBEDDING='{LABEL_EMB}'. Allowed: {list(EMBEDDING_SPECS)}")

MAX_SEQ_LEN: int = 512



# ────────────────────────────────────────────────────────────────────────────
# Per-embedding tables/plots
# ────────────────────────────────────────────────────────────────────────────
def _auto_flag(val: str, default_auto: bool) -> bool:
    v = (val or "").strip().lower()
    if v in {"1","true","yes","y"}:  return True
    if v in {"0","false","no","n"}:  return False
    return default_auto

MULTI_EMBED = len(STUDENT_EMBEDDINGS) > 1
WRITE_PER_EMBED_TABLES: bool = _auto_flag("auto", default_auto=not MULTI_EMBED)
WRITE_PER_EMBED_PLOTS:  bool = _auto_flag("auto", default_auto=not MULTI_EMBED)

if not WRITE_PER_EMBED_TABLES:
    log.info("[guard] Per-embedding tables disabled (combined report will handle tables).")
if not WRITE_PER_EMBED_PLOTS:
    log.info("[guard] Per-embedding plots disabled (combined report will draw plots).")

def write_run_config():
    """Dump a lightweight snapshot of env + resolved settings for reproducibility."""
    try:
        cfg = {
            "run_id": RUN_ID,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "device": DEVICE_STR,
            "seed": SEED,
            "n_jobs": N_JOBS,
            "student_embeddings": STUDENT_EMBEDDINGS,
            "current_student_embedding": STUDENT_EMB,
            "students": STUDENTS,
            "methods_filter_env": os.getenv("METHODS", "").strip() or None,
            "pct_list": PCT_LIST,
            "pcts_for_plots": PCTS_FOR_PLOTS,
            "label_source_embedding": LABEL_EMB,
            "label_source_model": LABEL_MODEL,
            "max_folds": MAX_FOLDS,
            "boot_B": BOOT_B,
            "write_per_embed_tables": WRITE_PER_EMBED_TABLES,
            "write_per_embed_plots": WRITE_PER_EMBED_PLOTS,
            "skip_baseline_if_present": os.getenv("SKIP_BASELINE_IF_PRESENT", "0"),
            "do_refits": os.getenv("DO_REFITS", "1"),
            "paths": {
                "root": str(ROOT),
                "g1_dir": str(G1_DIR),
                "g2_dir": str(G2_DIR),
                "g3a_dir": str(G3A_DIR),
                "results_dir": str(RES_DIR),
                "tables_dir": str(TABLES_DIR),
                "figs_dir": str(FIGS_DIR),
            },
        }
        (G3A_DIR / "run_config.json").write_text(json.dumps(cfg, indent=2), encoding="utf-8")
        log.info("run_config.json written.")
    except Exception as e:
        log.warning(f"Could not write run_config.json: {e}")

# ────────────────────────────────────────────────────────────────────────────
#  Sentence encoders
# ────────────────────────────────────────────────────────────────────────────
_ENCODERS: Dict[str, SentenceTransformer] = {}

def get_encoder(emb_key: str) -> SentenceTransformer:
    if emb_key in _ENCODERS:
        return _ENCODERS[emb_key]
    repo = EMBEDDING_SPECS[emb_key]
    log.info(f"Loading SentenceTransformer [{emb_key}]: {repo} → device={DEVICE_STR}")
    m = SentenceTransformer(repo, device=DEVICE_STR)
    m.max_seq_length = MAX_SEQ_LEN
    _ENCODERS[emb_key] = m
    return m

def encode_texts(emb_key: str, texts: List[str], batch_size: int = 64) -> np.ndarray:
    mdl = get_encoder(emb_key)
    X = mdl.encode(texts, batch_size=batch_size, convert_to_numpy=True,
                   show_progress_bar=False, normalize_embeddings=False)
    return X.astype(np.float32, copy=False)

# ────────────────────────────────────────────────────────────────────────────
#  Student models
# ────────────────────────────────────────────────────────────────────────────
class RegressorChainCV(BaseEstimator, RegressorMixin):
    """Cross-validated out-of-fold chaining with randomized target order support."""
    def __init__(self, base_estimator, order=None, cv_splits=5, random_state=SEED):
        self.base_estimator = base_estimator
        self.order = order
        self.cv_splits = cv_splits
        self.random_state = random_state
        self.chain_models_ = []
        self.n_targets_ = None

    def fit(self, X, Y):
        rng = check_random_state(self.random_state)
        n_samples, self.n_targets_ = Y.shape
        if self.order is None:
            self.order = np.arange(self.n_targets_)
        kf = KFold(n_splits=self.cv_splits, shuffle=True, random_state=rng)
        X_chain = np.ascontiguousarray(X, dtype=np.float32)

        oof_cols = []
        for target_idx in self.order:
            y = Y[:, target_idx]
            oof = np.zeros(n_samples, dtype=np.float32)
            for tr, va in kf.split(X_chain):
                est = clone(self.base_estimator)
                est.fit(X_chain[tr], y[tr])
                oof[va] = est.predict(X_chain[va])
            oof_cols.append(oof.reshape(-1, 1))
            X_chain = np.hstack([X_chain, oof.reshape(-1,1)])

        self.chain_models_ = []
        acc = []
        for i, target_idx in enumerate(self.order):
            if i == 0:
                X_full = X
            else:
                acc.append(oof_cols[i-1])
                X_full = np.hstack([X, np.hstack(acc)])
            est = clone(self.base_estimator)
            est.fit(X_full, Y[:, target_idx])
            self.chain_models_.append(est)
        return self

    def predict(self, X):
        X_ext = np.ascontiguousarray(X, dtype=np.float32)
        n = X.shape[0]
        preds = np.zeros((n, self.n_targets_), dtype=np.float32)
        for i, target_idx in enumerate(self.order):
            yhat = self.chain_models_[i].predict(X_ext).reshape(-1, 1)
            preds[:, target_idx] = yhat[:, 0]
            X_ext = np.hstack([X_ext, yhat])
        return preds

class EnsembleRegressorChainsCV(BaseEstimator, RegressorMixin):
    """Ensemble over random target orders; average predictions."""
    def __init__(self, base_estimator, n_chains=5, cv_splits=5, random_state=SEED):
        self.base_estimator = base_estimator
        self.n_chains = n_chains
        self.cv_splits = cv_splits
        self.random_state = random_state
        self.ensemble_ = None
        self.n_targets_ = None

    def fit(self, X, Y):
        rng = check_random_state(self.random_state)
        self.n_targets_ = Y.shape[1]
        self.ensemble_ = []
        for _ in range(self.n_chains):
            order = np.arange(self.n_targets_)
            rng.shuffle(order)
            chain = RegressorChainCV(
                base_estimator=self.base_estimator,
                order=order,
                cv_splits=self.cv_splits,
                random_state=rng.randint(0, 1_000_000),
            )
            chain.fit(X, Y)
            self.ensemble_.append((order, chain))
        return self

    def predict(self, X):
        preds = [chain.predict(X) for (_, chain) in self.ensemble_]
        return np.mean(preds, axis=0)

def build_with_hp(student_key: str, hp: Dict[str, float | int]) -> Pipeline:
    if student_key == "chain_ERCcv_lr":
        return Pipeline([
            ("pca", PCA(random_state=SEED, n_components=float(hp["pca__n_components"]))),
            ("chain", EnsembleRegressorChainsCV(
                base_estimator=LinearRegression(),
                n_chains=int(hp["chain__n_chains"]),
                cv_splits=int(hp["chain__cv_splits"]),
                random_state=SEED
            )),
        ])
    elif student_key == "local_lasso":
        return Pipeline([
            ("pca", PCA(random_state=SEED, n_components=float(hp["pca__n_components"]))),
            ("reg", MultiOutputRegressor(Lasso(alpha=float(hp["reg__estimator__alpha"]),
                                               random_state=SEED, max_iter=10_000))),
        ])
    elif student_key == "local_rf":
        return Pipeline([
            ("pca", PCA(random_state=SEED, n_components=float(hp["pca__n_components"]))),
            ("reg", MultiOutputRegressor(RandomForestRegressor(
                n_estimators=int(hp["reg__estimator__n_estimators"]),
                max_depth=None if (hp.get("reg__estimator__max_depth", None) in [None, "None"]) else int(hp["reg__estimator__max_depth"]),
                min_samples_leaf=int(hp.get("reg__estimator__min_samples_leaf", 1)),
                random_state=SEED, n_jobs=1
            ))),
        ])
    elif student_key == "global_rf":
        return Pipeline([
            ("pca", PCA(random_state=SEED, n_components=float(hp["pca__n_components"]))),
            ("reg", RandomForestRegressor(
                n_estimators=int(hp["reg__n_estimators"]),
                max_depth=None if (hp.get("reg__max_depth", None) in [None, "None"]) else int(hp["reg__max_depth"]),
                min_samples_leaf=int(hp.get("reg__min_samples_leaf", 1)),
                random_state=SEED, n_jobs=1
            )),
        ])
    elif student_key == "chain_ERCcv_rf":
        base_rf = RandomForestRegressor(
            n_estimators=int(hp.get("chain__base_rf__n_estimators", 100)),
            max_depth=None if (hp.get("chain__base_rf__max_depth", None) in [None, "None"]) else int(hp["chain__base_rf__max_depth"]),
            min_samples_leaf=int(hp.get("chain__base_rf__min_samples_leaf", 1)),
            random_state=SEED, n_jobs=1
        )
        return Pipeline([
            ("pca", PCA(random_state=SEED, n_components=float(hp["pca__n_components"]))),
            ("chain", EnsembleRegressorChainsCV(
                base_estimator=base_rf,
                n_chains=int(hp["chain__n_chains"]),
                cv_splits=int(hp["chain__cv_splits"]),
                random_state=SEED
            )),
        ])
    else:
        raise ValueError(f"Unknown student_key: {student_key}")

# ────────────────────────────────────────────────────────────────────────────
#  Load per-fold teacher pickle & extract HPs
# ────────────────────────────────────────────────────────────────────────────
def teacher_pickle_path(fold_idx: int, student_key: str, emb_key: str) -> Path:
    # output/g_synth_augmented/g_2a_teacher_labeling_loocv_tuned/teacher/teacher_fold{ii}_{emb}__{student}.pkl
    return ROOT / "models" / "teacher" / f"teacher_fold{fold_idx:02d}_{emb_key}__{student_key}.pkl"

def _hp_json_path(fold_idx: int, student_key: str, emb_key: str) -> Path:
    return G2_DIR / "teacher" / f"hp_fold{fold_idx:02d}_{emb_key}__{student_key}.json"

def hp_from_teacher(est: Pipeline, student_key: str) -> Dict[str, float | int]:
    hp: Dict[str, float | int] = {}

    if "pca" in est.named_steps:
        hp["pca__n_components"] = float(getattr(est.named_steps["pca"], "n_components", 0.8))

    if student_key == "chain_ERCcv_lr":
        ch = est.named_steps["chain"]
        hp["chain__n_chains"] = int(getattr(ch, "n_chains", 5))
        hp["chain__cv_splits"] = int(getattr(ch, "cv_splits", 5))

    elif student_key == "local_lasso":
        reg = est.named_steps["reg"].estimator
        hp["reg__estimator__alpha"] = float(getattr(reg, "alpha", 0.01))

    elif student_key == "local_rf":
        reg = est.named_steps["reg"].estimator
        hp["reg__estimator__n_estimators"] = int(getattr(reg, "n_estimators", 100))
        hp["reg__estimator__max_depth"] = getattr(reg, "max_depth", None)
        hp["reg__estimator__min_samples_leaf"] = int(getattr(reg, "min_samples_leaf", 1))

    elif student_key == "global_rf":
        reg = est.named_steps["reg"]
        hp["reg__n_estimators"] = int(getattr(reg, "n_estimators", 100))
        hp["reg__max_depth"] = getattr(reg, "max_depth", None)
        hp["reg__min_samples_leaf"] = int(getattr(reg, "min_samples_leaf", 1))

    elif student_key == "chain_ERCcv_rf":
        ch = est.named_steps["chain"]
        hp["chain__n_chains"] = int(getattr(ch, "n_chains", 5))
        hp["chain__cv_splits"] = int(getattr(ch, "cv_splits", 5))

        base_rf = getattr(ch, "base_estimator", None)
        if base_rf is None and hasattr(ch, "base_estimator_"):
            base_rf = ch.base_estimator_

        if isinstance(base_rf, RandomForestRegressor):
            hp["chain__base_rf__n_estimators"] = int(getattr(base_rf, "n_estimators", 100))
            hp["chain__base_rf__max_depth"] = getattr(base_rf, "max_depth", None)
            hp["chain__base_rf__min_samples_leaf"] = int(getattr(base_rf, "min_samples_leaf", 1))
    else:
        raise ValueError(student_key)

    return hp

def load_teacher_fold_estimator(fold_idx: int, student_key: str, emb_key: str, retries: int = 3, sleep_s: float = 1.5) -> Optional[Pipeline]:
    """Load fitted teacher pipeline with a few retries for flaky network/cloud files."""
    p = teacher_pickle_path(fold_idx, student_key, emb_key)
    for attempt in range(1, retries + 1):
        if not p.exists():
            break
        try:
            with open(p, "rb") as f:
                return pickle.load(f)
        except Exception as e:
            if "timed out" in str(e).lower() or isinstance(e, OSError):
                log.warning(f"Could not load teacher pickle ({p.name}) [attempt {attempt}/{retries}]: {e}")
                if attempt < retries:
                    time.sleep(sleep_s * attempt)
                    continue
            else:
                log.warning(f"Could not load teacher pickle ({p.name}): {e}")
            break
    return None

def hp_from_teacher_or_json(fold_idx: int, student_key: str, emb_key: str, est: Optional[Pipeline]) -> Dict[str, float | int]:
    """
    Extract HPs from teacher estimator if provided; else from the per-fold hp_*.json.
    If neither is available, RAISE an error (no hardcoded defaults).
    """
    # 1) Teacher pickle provided
    if est is not None:
        return hp_from_teacher(est, student_key)

    # 2) JSON fallback
    j = _hp_json_path(fold_idx, student_key, emb_key)
    if j.exists():
        try:
            payload = json.loads(j.read_text(encoding="utf-8"))
            # Some older JSONs may store the whole payload; accept either {"best_params": {...}} or the flat dict
            if isinstance(payload, dict) and "best_params" in payload and isinstance(payload["best_params"], dict):
                return payload["best_params"]
            if isinstance(payload, dict):
                return payload  # assume it already is the HP dict
        except Exception as e:
            raise RuntimeError(f"HP JSON unreadable: {j.name} | {e}") from e

    # 3) Hard stop
    raise RuntimeError(
        f"Missing per-fold HPs for fold={fold_idx}, student={student_key}, emb={emb_key}. "
        f"Expected either teacher pickle ({teacher_pickle_path(fold_idx, student_key, emb_key).name}) "
        f"or HP JSON ({_hp_json_path(fold_idx, student_key, emb_key).name})."
    )

# ────────────────────────────────────────────────────────────────────────────
#  Data loading (seeds + synthetics)
# ────────────────────────────────────────────────────────────────────────────
def load_seeds() -> pd.DataFrame:
    scores = pd.read_csv(DATA_DIR / "activity_scores.csv")
    acts   = pd.read_csv(DATA_DIR / "activities.csv")
    dm = scores.pivot(index="activity_id", columns="domain_id", values="score").reset_index()
    dm = dm.rename(columns=lambda x: f"domain{x}" if isinstance(x, (int, np.integer)) else x)
    dm = dm.merge(acts[["activity_id", "question"]], on="activity_id", how="left")
    dm = dm.rename(columns={"activity_id":"seed_id", "question":"text"})
    dm = dm.sort_values("seed_id").reset_index(drop=True)
    assert len(dm) == 96, f"Expected 96 seeds, got {len(dm)}"
    return dm[["seed_id","text", *TARGET_COLS]]

def ensure_seed_vectors(seeds_df: pd.DataFrame, emb_key: str) -> np.ndarray:
    vec_path = ROOT / "outputs" / "results" / f"{emb_key}_vectors.npy"
    meta_path = vec_path.with_suffix(".meta.json")

    def _write_meta(X: np.ndarray):
        try:
            meta = {
                "embedding": emb_key,
                "repo": EMBEDDING_SPECS[emb_key],
                "max_seq_len": int(MAX_SEQ_LEN),
                "n_rows": int(X.shape[0]),
                "n_dim": int(X.shape[1]),
            }
            meta_path.write_text(json.dumps(meta, indent=2), encoding="utf-8")
        except Exception:
            pass

    # Prefer reuse only if rows AND max_seq_len match
    if vec_path.exists():
        try:
            X = np.load(vec_path)
            ok_rows = (X.shape[0] == len(seeds_df))
            ok_len = False
            if meta_path.exists():
                meta = json.loads(meta_path.read_text(encoding="utf-8"))
                ok_len = int(meta.get("max_seq_len", MAX_SEQ_LEN)) == int(MAX_SEQ_LEN)
            # If no meta, force a one-time rebuild to stamp the file with the correct max_seq_len
            if ok_rows and ok_len:
                np.save(CACHE_DIR / f"X_seed_{emb_key}.npy", X)
                return X
            else:
                log.warning("[%s] Rebuilding seed vectors (rows_ok=%s, max_seq_len_ok=%s).",
                            emb_key, ok_rows, ok_len)
        except Exception as e:
            log.warning("[%s] Existing vectors unusable (%s); rebuilding.", emb_key, e)

    log.info("Computing %s embeddings for seeds …", emb_key)
    X = encode_texts(emb_key, seeds_df["text"].astype(str).tolist(), batch_size=64)
    np.save(vec_path, X)
    np.save(CACHE_DIR / f"X_seed_{emb_key}.npy", X)
    _write_meta(X)
    log.info("✔ Seed vectors saved → %s", vec_path.relative_to(ROOT))
    return X

def base_from_method_and_M(method: str, M: int) -> str:
    return f"n{M}_{method}"

def g2_label_file(fold_idx: int, method: str, M: int, label_emb: str, label_model: str) -> Path:
    base = base_from_method_and_M(method, M)
    preferred = G2_DIR / f"g2f_labels_fold{fold_idx:02d}_{base}__{label_emb}__{label_model}.csv"
    if preferred.exists():
        return preferred
    # fallback: any model with the label embedding (keeps label source embedding fixed)
    cand = list(G2_DIR.glob(f"g2f_labels_fold{fold_idx:02d}_{base}__{label_emb}__*.csv"))
    if cand:
        return cand[0]
    return preferred  # will raise later if missing

def discover_methods_and_M_from_g2(label_emb: str) -> Dict[str, List[int]]:
    pats = sorted(G2_DIR.glob(f"g2f_labels_fold00_n*_*__{label_emb}__*.csv"))
    rows: Dict[str, set] = {}
    for p in pats:
        m = re.match(rf"g2f_labels_fold00_n(\d+)_([A-Za-z0-9_]+)__{label_emb}__.*\.csv$", p.name)
        if not m: 
            continue
        M = int(m.group(1)); method = m.group(2)
        rows.setdefault(method, set()).add(M)
    return {k: sorted(v) for k, v in rows.items()}

def synth_cache_paths_from_g2c(g1_path: Path, emb_key: str) -> Tuple[Path, Path]:
    base = g1_path.stem
    m = re.match(r"g_final_(n\d+_[A-Za-z0-9_]+)$", base)
    if m:
        base = m.group(1)
    cache_dir = G2_DIR / "cache" / "synth_embeds"
    candidates = [
        (cache_dir / f"g_final_{base}__{emb_key}.npy",  cache_dir / f"g_final_{base}__index.csv"),
        (cache_dir / f"{base}__{emb_key}.npy",          cache_dir / f"{base}__index.csv"),
    ]
    for npy, idx in candidates:
        if npy.exists() and idx.exists():
            return npy, idx
    nearby = sorted(p.name for p in cache_dir.glob(f"*{base}*"))
    tried  = " ; ".join([npy.name for npy, _ in candidates] + [idx.name for _, idx in candidates])
    raise FileNotFoundError(
        f"Missing synth cache for base='{base}' and embedding='{emb_key}'. Tried: {tried}. "
        f"Found near-matches in cache: {nearby}"
    )

def g1_source_csv(method: str, M: int) -> Path:
    p = G1_DIR / f"g_final_n{M}_{method}.csv"
    if not p.exists():
        raise FileNotFoundError(f"Missing Script-A source: {p}")
    return p

def ensure_synth_cache(method: str, M: int, emb_key: str) -> Tuple[Path, Path]:
    """Ensure (npy, index.csv) exist for Script-A synthetics under the given embedding."""
    src_csv = g1_source_csv(method, M)
    base = src_csv.stem
    m = re.match(r"g_final_(n\d+_[A-Za-z0-9_]+)$", base)
    base = m.group(1) if m else base

    cache_dir = G2_DIR / "cache" / "synth_embeds"
    npy  = cache_dir / f"g_final_{base}__{emb_key}.npy"
    idx  = cache_dir / f"g_final_{base}__index.csv"
    if npy.exists() and idx.exists():
        return npy, idx

    df = pd.read_csv(src_csv)
    if "text" not in df.columns:
        raise ValueError(f"{src_csv.name}: missing 'text' column")
    texts = df["text"].astype(str).tolist()

    log.info(f"[cache] building synth embeds for method={method}, M={M}, emb={emb_key} …")
    Xs = encode_texts(emb_key, texts, batch_size=64).astype(np.float32, copy=False)
    cache_dir.mkdir(parents=True, exist_ok=True)
    np.save(npy, Xs)
    pd.DataFrame({"text": texts}).to_csv(idx, index=False)
    log.info(f"[cache] ✔ saved → {npy.relative_to(ROOT)} ; {idx.relative_to(ROOT)}")
    return npy, idx


# ────────────────────────────────────────────────────────────────────────────
#  Metrics 
# ────────────────────────────────────────────────────────────────────────────
def rmse(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sqrt(np.mean((a - b) ** 2, axis=0))

def rrmse_vs_dummy(y_true: np.ndarray, y_pred: np.ndarray, y_dummy: np.ndarray) -> np.ndarray:
    rm = rmse(y_true, y_pred)
    rd = rmse(y_true, y_dummy)
    return rm / np.maximum(rd, 1e-12)

def _fmt_dt(sec: float) -> str:
    m, s = divmod(sec, 60.0)
    h, m = divmod(m, 60.0)
    if h >= 1: return f"{int(h)}h{int(m):02d}m{s:04.1f}s"
    if m >= 1: return f"{int(m)}m{s:04.1f}s"
    return f"{s:0.2f}s"

def paired_cliffs_delta(full: np.ndarray, base: np.ndarray):
    diff = base - full  # positive when full improves (lower is better)
    n_pos = int(np.sum(diff > 0))
    n_neg = int(np.sum(diff < 0))
    n_zero = int(np.sum(diff == 0))
    denom = max(1, (n_pos + n_neg))
    delta = (n_pos - n_neg) / denom
    mag = ("negligible" if abs(delta) < 0.147 else
           "small"       if abs(delta) < 0.33  else
           "medium"      if abs(delta) < 0.474 else
           "large")
    return float(delta), n_pos, n_neg, n_zero, mag

def pct_to_K(pct: int, n_seeds: int = 96) -> int:
    return max(1, int(round((pct / 100.0) * n_seeds)))

def _dom_cols(df: pd.DataFrame) -> List[str]:
    return [c for c in df.columns if c.startswith(_DOM_PREFIX)]

def _global_median_from_file(p: Path) -> float:
    df = pd.read_csv(p)
    cols = _dom_cols(df)
    if not cols:
        raise RuntimeError(f"{p.name}: no '{_DOM_PREFIX}*' columns present.")
    return float(np.median(df[cols].to_numpy(dtype=np.float32).ravel()))

def _flat_arrays(p_base: Path, p_full: Path) -> Tuple[np.ndarray, np.ndarray]:
    db = pd.read_csv(p_base)
    df = pd.read_csv(p_full)
    cols = _dom_cols(db)
    if not cols or not set(cols).issubset(df.columns):
        missing = sorted(set(cols) - set(df.columns))
        raise RuntimeError(f"{p_full.name}: missing '{_DOM_PREFIX}*' columns: {missing}")
    xb = db[cols].to_numpy(dtype=np.float32).ravel()
    xf = df[cols].to_numpy(dtype=np.float32).ravel()
    return xb, xf

def section(title):
    """Print section header"""
    bar = "═" * len(title)
    print(f"\n{bar}\n{title}\n{bar}")

def _save_and_show(fig, path: str):
    """Save and display figure"""
    fig.savefig(path, bbox_inches="tight", dpi=300)
    plt.show()
    print(f"Plot saved → {path}")

def aligned_ranks(mat):
    """Hodges–Lehmann alignment + ranking along rows (lower is better)"""
    aligned = mat - np.median(mat, axis=1, keepdims=True)
    return np.apply_along_axis(lambda r: np.argsort(np.argsort(r)) + 1, 1, aligned)

def friedman_aligned(mat):
    """Aligned-Friedman χ² and Iman–Davenport F-statistic (expects ranks or aligned data)"""
    k = mat.shape[1]
    chi2, _ = friedmanchisquare(*[mat[:, i] for i in range(k)])
    Ff = ((mat.shape[0] - 1) * chi2) / (mat.shape[0] * (k - 1) - chi2)
    return chi2, Ff

def wilcoxon_matrix(mat, labels):
    """Pairwise two-sided Wilcoxon (zero-method = zsplit)"""
    df = pd.DataFrame(np.ones((len(labels), len(labels))), index=labels, columns=labels)
    for i, j in combinations(range(len(labels)), 2):
        diff = mat[:, i] - mat[:, j]
        p = 1.0 if np.allclose(diff, 0) else wilcoxon(diff, zero_method="zsplit")[1]
        df.iat[i, j] = df.iat[j, i] = p
    return df.round(4)

def holm_correct_and_effects(raw_p, data, labels):
    """Holm–Bonferroni correction and Cliff's Δ effect sizes"""
    idx = list(combinations(range(len(labels)), 2))
    pvals = [raw_p.iat[i, j] for i, j in idx]
    _, p_adj, _, _ = multipletests(pvals, method="holm")

    adj_df = raw_p.copy()
    for (i, j), p in zip(idx, p_adj):
        adj_df.iat[i, j] = adj_df.iat[j, i] = p
    adj_df[np.eye(len(labels), dtype=bool)] = 1.0

    def cliffs_delta(x, y):
        diffs = np.subtract.outer(x, y)
        n = len(x) * len(y)
        return (np.sum(diffs > 0) - np.sum(diffs < 0)) / n

    delta_df = pd.DataFrame(np.ones((len(labels), len(labels))), index=labels, columns=labels)
    for (i, j) in idx:
        d_ij = cliffs_delta(data[:, i], data[:, j])
        delta_df.iat[i, j] = d_ij
        delta_df.iat[j, i] = -d_ij

    return adj_df.round(4), delta_df.round(3)

def conover_posthoc(ranks, labels, fname_tag):
    """Conover–Iman test with Holm correction"""
    p_df = sp.posthoc_conover_friedman(ranks, p_adjust="holm")
    p_df.index = p_df.columns = labels
    out = TABLES_DIR / f"{fname_tag}_conover_p.csv"
    p_df.to_csv(out)
    print("\nConover–Iman post-hoc p-values (Holm-adjusted):")
    print(p_df.round(4).to_string())
    print("  ↳ saved →", out)
    return p_df

def run_friedman(mat, block_name, col_labels, fname_tag):
    """Generic routine for Friedman analysis with post-hoc tests"""
    k = len(col_labels)
    nblocks = mat.shape[0]

    # Save & print medians (PRINT SORTED low→high; CSV keeps original order)
    col_meds = pd.Series(np.median(mat, axis=0), index=col_labels)
    med_path = TABLES_DIR / f"{fname_tag}_median.csv"
    col_meds.to_csv(med_path, header=["median_rrmse"])
    print(f"\nMedian RRMSE per {block_name[:-1] if block_name.endswith('s') else block_name} (sorted low→high):")
    print(col_meds.sort_values().round(3).to_string())
    print("  ↳ saved →", med_path)

    if nblocks == 2:
        print(f"\nOnly two {block_name} → skipping Friedman/post-hoc.")
        wilc = wilcoxon_matrix(mat, col_labels)
        print("\nWilcoxon pairwise p-values:")
        print(wilc.round(4).to_string())
        wilc_path = TABLES_DIR / f"{fname_tag}_wilcoxon_raw_p.csv"
        wilc.to_csv(wilc_path)
        print("  ↳ saved →", wilc_path)

        adj, delta = holm_correct_and_effects(wilc, mat, col_labels)
        print("\nHolm–Bonferroni adjusted p-values:")
        print(adj.round(4).to_string())
        adj_path = TABLES_DIR / f"{fname_tag}_wilcoxon_holm_p.csv"
        adj.to_csv(adj_path)
        print("  ↳ saved →", adj_path)

        print("\nCliff's Δ effect sizes:")
        print(delta.round(3).to_string())
        delta_path = TABLES_DIR / f"{fname_tag}_cliffs_delta.csv"
        delta.to_csv(delta_path)
        print("  ↳ saved →", delta_path)
        return

    if k == 2:
        p = wilcoxon(mat[:, 0], mat[:, 1], zero_method="zsplit")[1]
        print(f"\nPaired Wilcoxon ({col_labels[0]} vs {col_labels[1]}): p = {p:.5g}")
        return

    # Friedman statistics
    ranks = aligned_ranks(mat)
    chi2_a, Ff_a = friedman_aligned(ranks)
    chi2_o, p_o = friedmanchisquare(*[mat[:, i] for i in range(k)])
    Ff_o = ((nblocks - 1) * chi2_o) / (nblocks * (k - 1) - chi2_o)

    print(f"\n*Aligned-Friedman* (blocks = {block_name})")
    print(f"  χ²_F = {chi2_a:.3f}    F_F = {Ff_a:.3f}")
    print(f"\n*Original-Friedman* (blocks = {block_name})")
    print(f"  χ²_F = {chi2_o:.3f}    p = {p_o:.3g}    F_F = {Ff_o:.3f}")

    # Post-hoc tests
    if nblocks < 10:
        conover_posthoc(ranks, col_labels, fname_tag)
    else:
        pvals_nem = sp.posthoc_nemenyi_friedman(ranks)
        pvals_nem.index = pvals_nem.columns = col_labels
        nem_path = TABLES_DIR / f"{fname_tag}_nemenyi_p.csv"
        pvals_nem.to_csv(nem_path)
        print("\nNemenyi p-values (aligned post-hoc):")
        print(pvals_nem.round(4).to_string())
        print("  ↳ saved →", nem_path)

    # 1) Pair-wise Wilcoxon
    wilc = wilcoxon_matrix(mat, col_labels)
    print("\nWilcoxon pairwise p-values:")
    print(wilc.round(4).to_string())
    wilc_path = TABLES_DIR / f"{fname_tag}_wilcoxon_raw_p.csv"
    wilc.to_csv(wilc_path)
    print("  ↳ saved →", wilc_path)

    # 2) Holm–Bonferroni adjustment + Cliff’s Δ
    adj, delta = holm_correct_and_effects(wilc, mat, col_labels)

    print("\nHolm–Bonferroni adjusted p-values:")
    print(adj.round(4).to_string())
    adj_path = TABLES_DIR / f"{fname_tag}_wilcoxon_holm_p.csv"
    adj.to_csv(adj_path)
    print("  ↳ saved →", adj_path)

    print("\nCliff's Δ effect sizes:")
    print(delta.round(3).to_string())
    delta_path = TABLES_DIR / f"{fname_tag}_cliffs_delta.csv"
    delta.to_csv(delta_path)
    print("  ↳ saved →", delta_path)

def vector_per_target(rrmse_array):
    """Collapse (folds × targets) → (targets,) by median across folds"""
    return np.median(rrmse_array, axis=0) if getattr(rrmse_array, "ndim", 1) == 2 else rrmse_array

def matrix_per_target_compare_models(data_dict, model_list, embedding_list):
    """Build matrix: rows = targets, cols = models, aggregated across embeddings"""
    return np.column_stack([
        np.median(
            np.concatenate([
                vector_per_target(data_dict[emb][model])
                for emb in embedding_list if emb in data_dict
            ], axis=0).reshape(-1, N_TARGETS),
            axis=0
        )
        for model in model_list
    ])

# --- PATCH: show full statistics right under each CD diagram -----------------

def cd_plot(matrix, labels, title, fname):
    """Critical-distance diagram with robust p-value alignment to labels."""
    if matrix.shape[1] < 2:
        print(f"⚠  Skipping CD-plot '{title}' (need ≥2 methods, got {matrix.shape[1]})")
        return

    ranks = aligned_ranks(matrix)

    # Compute post-hoc p-values and FORCE index/columns to match `labels`
    pvals_raw = sp.posthoc_nemenyi_friedman(ranks)
    if not isinstance(pvals_raw, pd.DataFrame):
        pvals = pd.DataFrame(pvals_raw, index=range(len(labels)), columns=range(len(labels)))
    else:
        pvals = pvals_raw.copy()

    # Defensive shape fix (trim/pad unlikely; trim covers rare inconsistencies)
    if pvals.shape != (len(labels), len(labels)):
        pvals = pvals.iloc[:len(labels), :len(labels)]
        if pvals.shape != (len(labels), len(labels)):
            # Last resort: identity p-values (no significant lines)
            pvals = pd.DataFrame(np.ones((len(labels), len(labels))), index=range(len(labels)), columns=range(len(labels)))

    # Align names to your model labels, sanitize & symmetrize
    pvals.index = labels
    pvals.columns = labels
    pvals = pvals.astype(float).fillna(1.0)
    pvals = pd.DataFrame(np.minimum(pvals.values, pvals.values.T), index=labels, columns=labels)
    np.fill_diagonal(pvals.values, 1.0)

    fig, ax = plt.subplots(figsize=(8, 3), dpi=120)
    sp.critical_difference_diagram(pd.Series(ranks.mean(0), index=labels), pvals, ax=ax)
    ax.set_title(title, pad=10)
    _save_and_show(fig, PLOTS_DIR / fname)

    # full report printed under the plot
    tag = Path(fname).stem
    run_friedman(matrix, block_name="folds", col_labels=labels, fname_tag=tag)


def cd_plot_dual(matrix1, labels1, matrix2, labels2, title1, title2, fname):
    """Two CD-diagrams side-by-side with robust p-value alignment."""
    if matrix1.shape[1] < 2 or matrix2.shape[1] < 2:
        print("⚠  Skipping dual CD-plot (need ≥2 methods for both)")
        return

    def _aligned_pvals(M, lbls):
        r = aligned_ranks(M)
        raw = sp.posthoc_nemenyi_friedman(r)
        if not isinstance(raw, pd.DataFrame):
            P = pd.DataFrame(raw, index=range(len(lbls)), columns=range(len(lbls)))
        else:
            P = raw.copy()
        if P.shape != (len(lbls), len(lbls)):
            P = P.iloc[:len(lbls), :len(lbls)]
            if P.shape != (len(lbls), len(lbls)):
                P = pd.DataFrame(np.ones((len(lbls), len(lbls))), index=range(len(lbls)), columns=range(len(lbls)))
        P.index = lbls
        P.columns = lbls
        P = P.astype(float).fillna(1.0)
        P = pd.DataFrame(np.minimum(P.values, P.values.T), index=lbls, columns=lbls)
        np.fill_diagonal(P.values, 1.0)
        return r, P

    ranks1, pvals1 = _aligned_pvals(matrix1, labels1)
    ranks2, pvals2 = _aligned_pvals(matrix2, labels2)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 3), dpi=120)
    sp.critical_difference_diagram(pd.Series(ranks1.mean(0), index=labels1), pvals1, ax=ax1)
    ax1.set_title(title1, pad=10)
    sp.critical_difference_diagram(pd.Series(ranks2.mean(0), index=labels2), pvals2, ax=ax2)
    ax2.set_title(title2, pad=10)
    plt.tight_layout()
    _save_and_show(fig, PLOTS_DIR / fname)

    base_tag = Path(fname).stem
    section(f"Full statistics — LEFT panel: {title1}")
    run_friedman(matrix1, block_name="folds", col_labels=labels1, fname_tag=f"{base_tag}__left")

    section(f"Full statistics — RIGHT panel: {title2}")
    run_friedman(matrix2, block_name="folds", col_labels=labels2, fname_tag=f"{base_tag}__right")


# ────────────────────────────────────────────────────────────────────────────
#  Reuse for chain_ERCcv_lr — only for e5_base student == label source
# ────────────────────────────────────────────────────────────────────────────
def chainlr_pct_full_path(student_emb: str, method: str, pct: int, K: int, Mmax: int) -> Path:
    return RES_DIR / f"rrmse_perfold_{student_emb}__chain_ERCcv_lr__{method}__pct{pct}_K{K}__Mmax{Mmax}__full.csv"

def ensure_chainlr_from_legacy_pct(student_emb: str, method: str, pct: int, K: int, N_SEEDS: int, Mmax: int) -> bool:
    if not (student_emb == "e5_base" and LABEL_EMB == "e5_base"):
        return False
    mapping = {100: ("n96_sps1", 96), 200: ("n192_sps2", 192), 400: ("n384_sps4", 384)}
    if pct not in mapping:
        return False
    tag, _ = mapping[pct]
    legacy_root = RES_DIR
    legacy_full = legacy_root / f"rrmse_perfold_e5_base__chain_ERCcv_lr__{method}__{tag}__full.csv"
    if not legacy_full.exists():
        log.info(f"[reuse/chain_lr] Missing legacy {legacy_full.name} → recompute instead.")
        return False

    new_full = chainlr_pct_full_path(student_emb, method, pct, K, Mmax)
    try:
        if new_full.exists():
            n_rows = sum(1 for _ in open(new_full, "r", encoding="utf-8")) - 1
            if n_rows >= N_SEEDS:
                return True
    except Exception:
        pass

    df_old = pd.read_csv(legacy_full).iloc[:N_SEEDS].copy()
    keep_cols = [c for c in df_old.columns if c.startswith("rrmse_domain")]
    if "median_rrmse_fold" in df_old.columns:
        keep_cols = ["median_rrmse_fold"] + keep_cols

    fold_col = df_old["fold"].astype(int).values if "fold" in df_old.columns else np.arange(N_SEEDS, dtype=int)
    df_new = pd.DataFrame({
        "fold": fold_col,
        "method": method,
        "pct": pct,
        "K": K,
        "M_max": Mmax,
        "student": "S1",
        "embedding": student_emb,
        "regressor": "chain_ERCcv_lr",
        "variant": "full",
    })
    for c in keep_cols:
        df_new[c] = df_old[c].values

    new_full.parent.mkdir(parents=True, exist_ok=True)
    df_new.to_csv(new_full, index=False)
    log.info(f"[reuse/chain_lr] Wrote {new_full.name} from legacy {legacy_full.name}")
    return True

def run_for_embedding(emb_key: str):
    """Run the whole pipeline for a single student embedding key."""
    global STUDENT_EMB
    if emb_key not in EMBEDDING_SPECS:
        raise ValueError(f"Unknown embedding '{emb_key}'. Allowed: {list(EMBEDDING_SPECS)}")
    prev = STUDENT_EMB
    try:
        STUDENT_EMB = emb_key
        log.info(f"=== BEGIN run() for STUDENT_EMBEDDING={emb_key} ===")
        run()
        log.info(f"=== END   run() for STUDENT_EMBEDDING={emb_key} ===")
    finally:
        STUDENT_EMB = prev

# ────────────────────────────────────────────────────────────────────────────
#  Combined reports 
# ────────────────────────────────────────────────────────────────────────────

def _matrix_for_method_pct(emb: str, method: str, pct: int, Mmax: int, regressors: List[str]) -> Tuple[np.ndarray, List[str]]:
    """
    Build mat of shape (n_folds, n_regressors) using FULL variant,
    each cell = median over domain-wise RRMSE for that fold.
    Only keeps folds present for ALL regressors (inner join on 'fold').
    """
    K = pct_to_K(pct, 96)
    fold_sets = []
    per_reg = {}

    for reg in regressors:
        p_full = RES_DIR / f"rrmse_perfold_{emb}__{reg}__{method}__pct{pct}_K{K}__Mmax{Mmax}__full.csv"
        if not p_full.exists():
            continue
        df = pd.read_csv(p_full)
        if "median_rrmse_fold" in df.columns:
            z = df[["fold", "median_rrmse_fold"]].rename(columns={"median_rrmse_fold":"med"})
        else:
            cols = [c for c in df.columns if c.startswith("rrmse_domain")]
            if not cols:
                continue
            z = df[["fold", *cols]].copy()
            z["med"] = z[cols].median(axis=1)
            z = z[["fold", "med"]]
        per_reg[reg] = z
        fold_sets.append(set(z["fold"].astype(int).tolist()))

    regs = sorted(per_reg.keys())
    if len(regs) < 2:
        return np.empty((0, 0)), []

    common_folds = sorted(set.intersection(*fold_sets)) if fold_sets else []
    if len(common_folds) < 2:
        return np.empty((0, 0)), []

    mats = []
    for reg in regs:
        z = per_reg[reg]
        z = z[z["fold"].isin(common_folds)].sort_values("fold")
        mats.append(z["med"].to_numpy(dtype=np.float32))

    mat = np.vstack(mats).T  # rows = folds, cols = regs
    return mat, regs

def say(msg: str):
    """Print once to screen and write once to run.log (no console duplicates)."""
    stream = getattr(sys, "__stdout__", sys.stdout)
    print(msg, file=stream, flush=True)  
    _say_logger.info(msg)          

def build_combined_reports_across_embeddings():
    """
    Aggregate results already computed on disk into a single set of
    tables/plots using the GLOBAL MEDIAN RRMSE (median over all folds×domains).
    No PRIMARY/Appendix split — one unified set of CSVs for all %K in PCT_LIST.
    """
    import matplotlib.pyplot as plt

    # Discover methods with optional filter
    avail = discover_methods_and_M_from_g2(LABEL_EMB)
    methods = sorted(avail.keys()) if (METHODS is None) else [m for m in sorted(avail.keys()) if m in METHODS]

    # Pattern for per-fold CSVs
    pat = re.compile(
        r"^rrmse_perfold_(?P<emb>.+?)__(?P<reg>.+?)__(?P<meth>.+?)__pct(?P<pct>\d+)_K(?P<K>\d+)__Mmax(?P<M>\d+)__(?P<var>baseline|full)\.csv$"
    )

    summary_rows, wil_rows, cliffs_rows = [], [], []
    pooled_pairs: Dict[Tuple[str,int], Dict[str, List[float]]] = {}

    # Scan all embeddings/regressors/methods/%K present on disk
    for emb in STUDENT_EMBEDDINGS:
        full_files = sorted([p for p in RES_DIR.glob(f"rrmse_perfold_{emb}__*__full.csv") if pat.match(p.name)])
        for f_full in full_files:
            m = pat.match(f_full.name)
            reg = m["reg"]; meth = m["meth"]
            pct = int(m["pct"]); K = int(m["K"]); Mmax = int(m["M"])
            if pct not in PCT_LIST:
                continue
            f_base = RES_DIR / f"rrmse_perfold_{emb}__{reg}__{meth}__pct{pct}_K{K}__Mmax{Mmax}__baseline.csv"
            if not f_base.exists():
                continue

            g_base = _global_median_from_file(f_base)
            g_full = _global_median_from_file(f_full)
            summary_rows.append({
                "student": "S1", "embedding": emb, "regressor": reg, "method": meth,
                "pct": pct, "K": K, "M_max": Mmax,
                "baseline": g_base, "full": g_full
            })

            # Paired tests on flattened (fold×domain) arrays
            xb, xf = _flat_arrays(f_base, f_full)
            pooled_pairs.setdefault((meth, pct), {"base": [], "full": []})
            pooled_pairs[(meth, pct)]["base"].extend(xb.tolist())
            pooled_pairs[(meth, pct)]["full"].extend(xf.tolist())

            try:
                _, p_raw = wilcoxon(xf, xb, zero_method="pratt", alternative="less")
            except Exception:
                p_raw = 1.0
            d, npos, nneg, nzero, mag = paired_cliffs_delta(xf, xb)
            wil_rows.append({"student":"S1","embedding":emb,"regressor":reg,"method":meth,
                             "pct":pct,"K":K,"M_max":Mmax,"p_raw":float(p_raw)})
            cliffs_rows.append({"student":"S1","embedding":emb,"regressor":reg,"method":meth,
                                "pct":pct,"K":K,"M_max":Mmax,"cliffs_delta_paired":float(d),
                                "npos":int(npos),"nneg":int(nneg),"nzero":int(nzero),"magnitude":mag})

    # ── Unified wide summary (all pcts)
    if not summary_rows:
        say("[combined] No result pairs found to summarise.")
        return

    wide = pd.DataFrame(summary_rows).sort_values(["embedding","regressor","method","pct"]).reset_index(drop=True)

    tol = 1e-9
    def _cmp(row):
        b, f = row["baseline"], row["full"]
        if pd.isna(b) or pd.isna(f): return "n/a"
        if (b - f) > tol:  return "better"
        if (f - b) > tol:  return "worse"
        return "same"
    wide["baseline_vs_full"] = wide.apply(_cmp, axis=1)

    # Save unified tables (no appendix split) — ALWAYS write combined outputs
    out_sum = TABLES_DIR / "summary_median_rrmse.csv"
    wide.to_csv(out_sum, index=False)
    say("[combined] summary_median_rrmse.csv written (unified, all pcts).")

    comp = wide.copy()
    comp["delta"] = comp["baseline"] - comp["full"]
    comp["rel_change_pct"] = 100.0 * comp["delta"] / comp["baseline"].replace(0, np.nan)
    comp.to_csv(TABLES_DIR / "summary_median_rrmse_with_delta.csv", index=False)
    say("[combined] summary_median_rrmse_with_delta.csv written (unified).")

    # Wilcoxon + Holm — single file for the combined set
    if wil_rows:
        df_w = pd.DataFrame(wil_rows)
        holm_rows = []
        for (embedding, regressor, method), grp in df_w.groupby(["embedding","regressor","method"]):
            pvals = grp["p_raw"].to_numpy()
            _, p_holm, _, _ = multipletests(pvals, method="holm")
            for i, (_, row) in enumerate(grp.reset_index(drop=True).iterrows()):
                holm_rows.append({
                    "student":"S1","embedding":embedding,"regressor":regressor,"method":method,
                    "pct": int(row["pct"]), "K": int(row["K"]), "M_max": int(row["M_max"]),
                    "p_raw": float(row["p_raw"]), "p_holm": float(p_holm[i]),
                })
        pd.DataFrame(holm_rows).sort_values(["embedding","regressor","method","pct"]).to_csv(
            TABLES_DIR / "wilcoxon_holm_vs_baseline.csv", index=False
        )
        say("[combined] wilcoxon_holm_vs_baseline.csv written.")

    # Cliff’s Δ — single file for the combined set
    if cliffs_rows:
        pd.DataFrame(cliffs_rows).sort_values(["embedding","regressor","method","pct"]).to_csv(
            TABLES_DIR / "cliffs_delta_vs_baseline.csv", index=False
        )
        say("[combined] cliffs_delta_vs_baseline.csv written.")

    # Pooled tests across (method, pct) — single files
    pooled_wil, pooled_cliffs = [], []
    for (meth, pct), bins in pooled_pairs.items():
        xb = np.asarray(bins["base"], dtype=np.float32)
        xf = np.asarray(bins["full"], dtype=np.float32)
        if xb.size and xb.shape == xf.shape:
            try:
                _, p_raw = wilcoxon(xf, xb, zero_method="pratt", alternative="less")
            except Exception:
                p_raw = 1.0
            d, npos, nneg, nzero, mag = paired_cliffs_delta(xf, xb)
            pooled_wil.append({"method": meth, "pct": pct, "n_pairs": int(xb.size), "p_value": float(p_raw)})
            pooled_cliffs.append({"method": meth, "pct": pct, "n_pairs": int(xb.size),
                                  "cliffs_delta_paired": float(d), "npos": int(npos),
                                  "nneg": int(nneg), "nzero": int(nzero), "magnitude": mag})
    if pooled_wil:
        pd.DataFrame(pooled_wil).sort_values(["method","pct"]).to_csv(
            TABLES_DIR / "wilcoxon_pooled_vs_baseline.csv", index=False
        )
        pd.DataFrame(pooled_cliffs).sort_values(["method","pct"]).to_csv(
            TABLES_DIR / "cliffs_delta_pooled_vs_baseline.csv", index=False
        )
        say("[combined] pooled tests written.")

    # Bootstrap Δ median (global micro-bootstrap over folds×domains) for ALL rows
    boots = []
    for _, row in wide.iterrows():
        emb, reg, meth = row["embedding"], row["regressor"], row["method"]
        pct, K, Mmax   = int(row["pct"]), int(row["K"]), int(row["M_max"])
        p_b = RES_DIR / f"rrmse_perfold_{emb}__{reg}__{meth}__pct{pct}_K{K}__Mmax{Mmax}__baseline.csv"
        p_f = RES_DIR / f"rrmse_perfold_{emb}__{reg}__{meth}__pct{pct}_K{K}__Mmax{Mmax}__full.csv"
        if not (p_b.exists() and p_f.exists()):
            continue
        db, df = pd.read_csv(p_b), pd.read_csv(p_f)
        cols = _dom_cols(db)
        if not cols or not set(cols).issubset(df.columns):
            continue
        base_mat = db[cols].to_numpy(dtype=np.float32)
        full_mat = df[cols].to_numpy(dtype=np.float32)

        rng = np.random.default_rng(SEED)
        n_folds, n_dom = base_mat.shape
        draws = np.empty(BOOT_B, dtype=np.float32)
        for b in range(BOOT_B):
            idx_f = rng.integers(0, n_folds, size=n_folds)
            idx_d = rng.integers(0, n_dom,   size=n_dom)
            b_flat = base_mat[idx_f][:, idx_d].ravel()
            f_flat = full_mat[idx_f][:, idx_d].ravel()
            draws[b] = np.median(b_flat) - np.median(f_flat)
        ci_lo, ci_hi = np.percentile(draws, [2.5, 97.5])
        boots.append({
            "student":"S1","embedding":emb,"regressor":reg,"method":meth,
            "pct":pct,"K":K,"M_max":Mmax,
            "delta_hat": float(np.median(draws)),
            "ci_lo": float(ci_lo), "ci_hi": float(ci_hi),
            "p_win": float((draws > 0).mean()), "B": int(BOOT_B)
        })
    if boots:
        pd.DataFrame(boots).sort_values(["embedding","regressor","method","pct"]).to_csv(
            TABLES_DIR / "bootstrap_delta_ci.csv", index=False
        )
        say("[combined] bootstrap_delta_ci.csv written.")

    # ── Combined plots (full range, baseline dotted, full solid; same color per (emb,reg))
    def _color_map_for_pairs(pairs):
        colors = plt.rcParams['axes.prop_cycle'].by_key().get('color', [])
        if not colors: colors = [f"C{i}" for i in range(10)]
        cyc = iter(colors * ((len(pairs) // max(1,len(colors))) + 2))
        return {pair: next(cyc) for pair in pairs}

    def plot_rrmse_vs_pct(summary_csv_path, methods_filter=None):
        df = pd.read_csv(summary_csv_path)
        need = {"embedding","regressor","method","pct","baseline","full"}
        if not need.issubset(df.columns):
            log.warning("[combined] summary file missing required columns; skipping plots.")
            return
        if methods_filter:
            df = df[df["method"].isin(methods_filter)].copy()

        for method in sorted(df["method"].unique()):
            sub = df[df["method"] == method].copy()
            agg = sub.groupby(["embedding","regressor","pct"], as_index=False)[["baseline","full"]].median()
            pairs = sorted(agg[["embedding","regressor"]].drop_duplicates().itertuples(index=False, name=None))
            cmap = _color_map_for_pairs(pairs)

            fig, ax = plt.subplots(figsize=(16, 4.8))
            xticks = sorted(set(PCTS_FOR_PLOTS) | set(agg["pct"].unique()))
            ax.set_xticks(xticks)
            ax.set_xlim(min(PCTS_FOR_PLOTS) - 5, max(PCTS_FOR_PLOTS) + 20)

            for emb, reg in pairs:
                cur = agg[(agg["embedding"] == emb) & (agg["regressor"] == reg)].sort_values("pct")
                color = cmap[(emb, reg)]
                ax.plot(cur["pct"], cur["full"],     marker="o", linestyle="-",  label=f"{emb} | {reg} (full)",  color=color)
                ax.plot(cur["pct"], cur["baseline"], marker="o", linestyle="--", label=f"{emb} | {reg} (baseline)", color=color)

            ax.set_xlabel("%K"); ax.set_ylabel("RRMSE")
            ax.set_title(f"(Global median) RRMSE vs %K — {method}")
            ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False, ncol=1)
            fig.tight_layout(rect=[0, 0, 0.80, 1])
            out = FIGS_DIR / f"combined_rrmse_vs_pct__{method}.png"
            fig.savefig(out, dpi=200, bbox_inches="tight")
            plt.close(fig)

    def plot_delta_vs_pct(summary_csv_path, methods_filter=None):
        df = pd.read_csv(summary_csv_path)
        need = {"embedding","regressor","method","pct","baseline","full"}
        if not need.issubset(df.columns):
            log.warning("[combined] summary file missing required columns; skipping Δ plot.")
            return
        if methods_filter:
            df = df[df["method"].isin(methods_filter)].copy()
        df["delta"] = df["baseline"] - df["full"]

        for method in sorted(df["method"].unique()):
            sub = df[df["method"] == method].copy()
            agg = sub.groupby(["embedding","regressor","pct"], as_index=False)["delta"].median()
            pairs = sorted(agg[["embedding","regressor"]].drop_duplicates().itertuples(index=False, name=None))
            cmap = _color_map_for_pairs(pairs)

            fig, ax = plt.subplots(figsize=(16, 4.8))
            xticks = sorted(set(PCTS_FOR_PLOTS) | set(agg["pct"].unique()))
            ax.set_xticks(xticks)
            ax.set_xlim(min(PCTS_FOR_PLOTS) - 5, max(PCTS_FOR_PLOTS) + 20)

            for emb, reg in pairs:
                cur = agg[(agg["embedding"] == emb) & (agg["regressor"] == reg)].sort_values("pct")
                color = cmap[(emb, reg)]
                ax.plot(cur["pct"], cur["delta"], marker="o", linestyle="-", label=f"{emb} | {reg}", color=color)

            ax.axhline(0.0, linestyle="--")
            ax.set_xlabel("%K"); ax.set_ylabel("ΔRRMSE (baseline - full)")
            ax.set_title(f"(Global median) ΔRRMSE vs %K — {method}")
            ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False, ncol=1)
            fig.tight_layout(rect=[0, 0, 0.80, 1])
            out = FIGS_DIR / f"combined_delta_vs_pct__{method}.png"
            fig.savefig(out, dpi=200, bbox_inches="tight")
            plt.close(fig)

    # Draw full-range combined plots 
    plot_rrmse_vs_pct(TABLES_DIR / "summary_median_rrmse.csv", methods_filter=methods)
    plot_delta_vs_pct(TABLES_DIR / "summary_median_rrmse.csv", methods_filter=methods)

    # --- CD diagrams + full printouts per (embedding, method, pct) ---
    for emb in STUDENT_EMBEDDINGS:
        for method in methods:
            Mmax = max(avail[method])  # same M_max you used elsewhere
            for pct in PCT_LIST:
                mat, labels = _matrix_for_method_pct(
                    emb, method, pct, Mmax,
                    regressors=sorted(df["regressor"].unique()) if 'df' in locals() else STUDENTS
                )
                if mat.size == 0 or len(labels) < 2:
                    continue
                tag = f"{method}__pct{pct}__{emb}__full"
                title = f"CD (global median per fold) — {method} @ {pct}%K • {emb} (full)"
                cd_plot(mat, labels, title, f"cd_{tag}.png")
                
# ────────────────────────────────────────────────────────────────────────────
#  Main functions
# ────────────────────────────────────────────────────────────────────────────
def _all_baselines_exist(students, methods, pct_list, N_SEEDS, student_emb, M_MAX_BY_METHOD):
    """Return True if for every (student, method, pct) the baseline CSV exists with >= N_SEEDS rows."""
    for method in methods:
        Mmax = M_MAX_BY_METHOD[method]
        for pct in pct_list:
            K = pct_to_K(pct, N_SEEDS)
            for student in students:
                out_base = RES_DIR / f"rrmse_perfold_{student_emb}__{student}__{method}__pct{pct}_K{K}__Mmax{Mmax}__baseline.csv"
                if not out_base.exists():
                    return False
                try:
                    n_rows = sum(1 for _ in open(out_base, "r", encoding="utf-8")) - 1
                    if n_rows < N_SEEDS:
                        return False
                except Exception:
                    return False
    return True

def run():
    log.info(f"=== step e_3 (LOOCV, percent; HPs from teacher, no tuning) | run_id={RUN_ID} ===")

    # ── Seeds
    seeds_df = load_seeds()
    X_seed = ensure_seed_vectors(seeds_df, STUDENT_EMB)
    Y_seed = seeds_df[TARGET_COLS].to_numpy(dtype=np.float32)
    N_SEEDS = len(seeds_df)
    folds = list(range(N_SEEDS))
    if MAX_FOLDS and MAX_FOLDS < N_SEEDS:
        folds = folds[:MAX_FOLDS]
        log.info(f"Using first {len(folds)} LOOCV folds (MAX_FOLDS={MAX_FOLDS})")
    else:
        log.info(f"Folds: {len(folds)} (LOOCV)")

    # ── Methods & M_max
    avail = discover_methods_and_M_from_g2(LABEL_EMB)
    if METHODS:
        avail = {m: avail[m] for m in avail if m in METHODS}
    if not avail:
        raise RuntimeError("No methods discovered in tuned g_2a_teacher_labeling (fold00 files).")
    methods = sorted(avail.keys())
    M_MAX_BY_METHOD = {m: max(avail[m]) for m in methods}
    log.info(f"Methods: {methods}")
    log.info(f"M_max per method: {M_MAX_BY_METHOD}")

    # ── Preload synth embeddings & index mappings for M_max
    SYN: Dict[Tuple[str,int], Dict[str, object]] = {}
    for method in methods:
        Mmax = M_MAX_BY_METHOD[method]
        src_csv = g1_source_csv(method, Mmax)
        try:
            npy, idx = synth_cache_paths_from_g2c(src_csv, STUDENT_EMB)
        except FileNotFoundError:
            npy, idx = ensure_synth_cache(method, Mmax, STUDENT_EMB)
        Xs = np.load(npy).astype(np.float32, copy=False)
        df_idx = pd.read_csv(idx)
        if "text" not in df_idx.columns:
            raise ValueError(f"{idx.name}: missing 'text' column")
        texts = df_idx["text"].astype(str).tolist()
        text2pos = {t: i for i, t in enumerate(texts)}
        SYN[(method, Mmax)] = {"X": Xs, "text2pos": text2pos}
        log.info(f"[{method}] M_max={Mmax}: loaded synth embeds ({Xs.shape[0]} rows)")

    # ── Preload per-fold tuned labels (teacher outputs) for M_max
    LABELS: Dict[Tuple[str,int], List[pd.DataFrame]] = {}
    label_cols = ["text", *TARGET_COLS]
    for method in methods:
        Mmax = M_MAX_BY_METHOD[method]
        per_fold = []
        for fi in folds:
            path = g2_label_file(fi, method, Mmax, label_emb=LABEL_EMB, label_model=LABEL_MODEL)
            if not path.exists():
                raise FileNotFoundError(f"Missing tuned labels: {path.name}")
            df = pd.read_csv(path, usecols=lambda c: (c == "text") or (c in TARGET_COLS))
            if set(label_cols).issubset(df.columns):
                df = df[label_cols].reset_index(drop=True)
            else:
                missing = [c for c in label_cols if c not in df.columns]
                raise ValueError(f"{path.name}: missing columns {missing}")
            per_fold.append(df)
        LABELS[(method, Mmax)] = per_fold
        log.info(f"[{method}] M_max={Mmax}: loaded tuned labels for {len(per_fold)} folds")

    # ── Baseline: seeds-only prediction (reuse teacher HPs where possible)
    SKIP_BASELINE_IF_PRESENT = os.getenv("SKIP_BASELINE_IF_PRESENT", "0").lower() in {"1","true","yes"}

    if SKIP_BASELINE_IF_PRESENT and _all_baselines_exist(STUDENTS, methods, PCT_LIST, N_SEEDS, STUDENT_EMB, M_MAX_BY_METHOD):
        log.info("[baseline] All baseline CSVs already present → skipping baseline computation.")
        baseline_core_by_student: Dict[str, List[Dict]] = {stu: [] for stu in STUDENTS}  # ensure writer no-ops
    else:
        baseline_core_by_student: Dict[str, List[Dict]] = {stu: [] for stu in STUDENTS}
        for fi in folds:
            test_mask = (np.arange(N_SEEDS) == fi)
            train_mask = ~test_mask
            X_tr_seed, Y_tr_seed = X_seed[train_mask], Y_seed[train_mask]
            X_te_seed, Y_te_seed = X_seed[test_mask],  Y_seed[test_mask]
            dummy = Y_tr_seed.mean(axis=0, keepdims=True)

            for stu in STUDENTS:
                est = load_teacher_fold_estimator(fi, stu, STUDENT_EMB)
                if est is not None:
                    Y_hat = est.predict(X_te_seed)
                else:
                    hp = hp_from_teacher_or_json(fi, stu, STUDENT_EMB, est=None)
                    model = build_with_hp(stu, hp)
                    model.fit(X_tr_seed, Y_tr_seed)
                    Y_hat = model.predict(X_te_seed)

                r = rrmse_vs_dummy(Y_te_seed, Y_hat, dummy)
                baseline_core_by_student[stu].append({
                    "fold": fi,
                    "median_rrmse_fold": float(np.median(r)),  # traceability
                    **{f"rrmse_{c}": float(r[i]) for i, c in enumerate(TARGET_COLS)}
                })

            if (fi + 1) % LOG_EVERY == 0:
                log.info(f"[baseline] folds completed: {fi+1}/{len(folds)}")


    # Helper: write/ensure per-pct baseline CSV once for naming symmetry
    def ensure_baseline_csv(student: str, method: str, pct: int, K: int, Mmax: int, out_base: Path):
        # If we didn’t compute baseline in this run, we can’t write fresh rows
        cores = baseline_core_by_student.get(student, [])
        if not cores:
            return

        rows = []
        for core in cores:
            rows.append({
                "fold": core["fold"],
                "method": method, "pct": pct, "K": K, "M_max": Mmax,
                "student": "S1", "embedding": STUDENT_EMB, "regressor": student, "variant": "baseline",
                "median_rrmse_fold": core["median_rrmse_fold"],
                **{k: core[k] for k in core if k.startswith("rrmse_domain")}
            })
        df_new = pd.DataFrame(rows)

        if out_base.exists():
            try:
                df_old = pd.read_csv(out_base)
                df_new = (pd.concat([df_old, df_new], axis=0, ignore_index=True)
                            .sort_values("fold")
                            .drop_duplicates(subset=["fold"], keep="last"))
            except Exception:
                pass

        out_base.parent.mkdir(parents=True, exist_ok=True)
        df_new.to_csv(out_base, index=False)



    # ── Full (augmented): per-fold HP reuse per student (no tuning)
    log.info("── Full (augmented): per-fold HP reuse (no tuning) for each student×method×pct")
    PCT_TO_K = {p: pct_to_K(p, N_SEEDS) for p in PCT_LIST}

    for method in methods:
        Mmax = M_MAX_BY_METHOD[method]
        for pct in PCT_LIST:
            K = PCT_TO_K[pct]
            for student in STUDENTS:
                out_full = RES_DIR / f"rrmse_perfold_{STUDENT_EMB}__{student}__{method}__pct{pct}_K{K}__Mmax{Mmax}__full.csv"
                out_base = RES_DIR / f"rrmse_perfold_{STUDENT_EMB}__{student}__{method}__pct{pct}_K{K}__Mmax{Mmax}__baseline.csv"

                # If FULL already complete, just ensure baseline and continue
                full_complete = False
                if out_full.exists():
                    try:
                        n_rows = sum(1 for _ in open(out_full, "r", encoding="utf-8")) - 1
                        full_complete = (n_rows >= len(folds))
                    except Exception:
                        full_complete = False
                if full_complete:
                    ensure_baseline_csv(student, method, pct, K, Mmax, out_base)
                    log.info(f"[skip] {out_full.name} already complete.")
                    continue

                # Legacy reuse for chain_ERCcv_lr only at {100,200,400}
                if student == "chain_ERCcv_lr" and pct in {100, 200, 400}:
                    if ensure_chainlr_from_legacy_pct(STUDENT_EMB, method, pct, K, N_SEEDS, Mmax):
                        ensure_baseline_csv(student, method, pct, K, Mmax, out_base)
                        continue

                # Compute missing folds
                pf_rows = []
                done_folds = set()
                if out_full.exists():
                    try:
                        done_folds = set(pd.read_csv(out_full, usecols=["fold"])["fold"].astype(int).tolist())
                    except Exception:
                        done_folds = set()

                for fi in folds:
                    if fi in done_folds:
                        continue
                    test_mask = (np.arange(N_SEEDS) == fi)
                    train_mask = ~test_mask
                    X_tr_seed, Y_tr_seed = X_seed[train_mask], Y_seed[train_mask]
                    X_te_seed, Y_te_seed = X_seed[test_mask],  Y_seed[test_mask]
                    dummy = Y_tr_seed.mean(axis=0, keepdims=True)

                    # fold-i labels on Mmax, slice first K deterministically
                    df_lab = LABELS[(method, Mmax)][fi].iloc[:K].reset_index(drop=True)
                    text_list = df_lab["text"].astype(str).tolist()
                    mapper = SYN[(method, Mmax)]["text2pos"]
                    pos = [mapper.get(t, -1) for t in text_list]
                    if any(i < 0 for i in pos):
                        miss = [t for t, i in zip(text_list, pos) if i < 0][:3]
                        raise RuntimeError(f"[{method}|pct={pct}|K={K}|fold={fi}] {len(miss)} label texts not in synth index. e.g., {miss}")
                    X_syn = SYN[(method, Mmax)]["X"][pos, :]
                    Y_syn = df_lab[TARGET_COLS].to_numpy(dtype=np.float32)

                    X_tr_full = np.ascontiguousarray(np.vstack([X_tr_seed, X_syn]), dtype=np.float32)
                    Y_tr_full = np.ascontiguousarray(np.vstack([Y_tr_seed, Y_syn]), dtype=np.float32)

                    # Load teacher HPs for THIS student & fold
                    est = load_teacher_fold_estimator(fi, student, STUDENT_EMB)
                    hp = hp_from_teacher_or_json(fi, student, STUDENT_EMB, est)

                    # Persist HPs for traceability
                    hp_file = HP_PERFOLD / student / method / f"n{Mmax}" / f"pct{pct}_K{K}" / f"fold{fi:02d}_best.json"
                    hp_file.parent.mkdir(parents=True, exist_ok=True)
                    try:
                        hp_file.write_text(json.dumps(hp, indent=2), encoding="utf-8")
                    except Exception:
                        pass

                    # Build model, fit on augmented train, predict held-out seed
                    model_full = build_with_hp(student, hp)
                    t0 = time.perf_counter()
                    model_full.fit(X_tr_full, Y_tr_full)
                    Y_hat_full = model_full.predict(X_te_seed)
                    dt_full = time.perf_counter() - t0

                    r_full = rrmse_vs_dummy(Y_te_seed, Y_hat_full, dummy)
                    pf_rows.append({
                        "fold": fi,
                        "method": method, "pct": pct, "K": K, "M_max": Mmax,
                        "student": "S1", "embedding": STUDENT_EMB, "regressor": student, "variant": "full",
                        "median_rrmse_fold": float(np.median(r_full)),  # trace only
                        **{f"rrmse_{c}": float(r_full[i]) for i, c in enumerate(TARGET_COLS)}
                    })

                    if (fi + 1) % LOG_EVERY == 0:
                        log.info(f"[{student}|{method}|pct={pct}|K={K}] folds done: {fi+1}/{len(folds)} (last full fit+pred {_fmt_dt(dt_full)})")

                # Write/append FULL; ensure BASELINE exists
                if pf_rows:
                    if out_full.exists():
                        df_old = pd.read_csv(out_full)
                        df_new = pd.concat([df_old, pd.DataFrame(pf_rows)], axis=0, ignore_index=True)
                    else:
                        df_new = pd.DataFrame(pf_rows)
                    df_new = df_new.sort_values("fold").drop_duplicates(subset=["fold"], keep="last")
                    out_full.parent.mkdir(parents=True, exist_ok=True)
                    df_new.to_csv(out_full, index=False)

                ensure_baseline_csv(student, method, pct, K, Mmax, out_base)

    # ── Unified (global-median) tables/stats/plots 

    pat = re.compile(
        r"^rrmse_perfold_(?P<emb>.+?)__(?P<reg>.+?)__(?P<meth>.+?)__pct(?P<pct>\d+)_K(?P<K>\d+)__Mmax(?P<M>\d+)__(?P<var>baseline|full)\.csv$"
    )
    summary_rows, wil_rows, cliffs_rows = [], [], []
    pooled_pairs: Dict[Tuple[str,int], Dict[str, List[float]]] = {}

    full_files = sorted([p for p in RES_DIR.glob(f"rrmse_perfold_{STUDENT_EMB}__*__full.csv") if pat.match(p.name)])
    for f_full in full_files:
        m = pat.match(f_full.name)
        reg = m["reg"]; meth = m["meth"]; pct = int(m["pct"]); K = int(m["K"]); Mmax = int(m["M"])
        if pct not in PCT_LIST:
            continue
        f_base = RES_DIR / f"rrmse_perfold_{STUDENT_EMB}__{reg}__{meth}__pct{pct}_K{K}__Mmax{Mmax}__baseline.csv"
        if not f_base.exists():
            continue

        g_base = _global_median_from_file(f_base)
        g_full = _global_median_from_file(f_full)
        summary_rows.append({
            "student":"S1","embedding":STUDENT_EMB,"regressor":reg,"method":meth,
            "pct":pct,"K":K,"M_max":Mmax, "baseline": g_base, "full": g_full
        })

        xb, xf = _flat_arrays(f_base, f_full)
        pooled_pairs.setdefault((meth, pct), {"base": [], "full": []})
        pooled_pairs[(meth, pct)]["base"].extend(xb.tolist())
        pooled_pairs[(meth, pct)]["full"].extend(xf.tolist())

        try:
            _, p_raw = wilcoxon(xf, xb, zero_method="pratt", alternative="less")
        except Exception:
            p_raw = 1.0
        d, npos, nneg, nzero, mag = paired_cliffs_delta(xf, xb)
        wil_rows.append({"student":"S1","regressor":reg,"method":meth,
                         "pct":pct,"K":K,"M_max":Mmax,"p_raw":float(p_raw)})
        cliffs_rows.append({"student":"S1","regressor":reg,"method":meth,
                            "pct":pct,"K":K,"M_max":Mmax,"cliffs_delta_paired":float(d),
                            "npos":int(npos),"nneg":int(nneg),"nzero":int(nzero),"magnitude":mag})

    if summary_rows:
        wide = pd.DataFrame(summary_rows).sort_values(["regressor","method","pct"]).reset_index(drop=True)
        tol = 1e-9
        def _cmp(row):
            b, f = row["baseline"], row["full"]
            if pd.isna(b) or pd.isna(f): return "n/a"
            if (b - f) > tol:  return "better"
            if (f - b) > tol:  return "worse"
            return "same"
        wide["baseline_vs_full"] = wide.apply(_cmp, axis=1)

        # Per-embedding tables (optional) — use embedding-suffixed filenames
        if WRITE_PER_EMBED_TABLES:
            out_sum_embed = TABLES_DIR / f"summary_median_rrmse__{STUDENT_EMB}.csv"
            wide.to_csv(out_sum_embed, index=False)

            comp = wide.copy()
            comp["delta"] = comp["baseline"] - comp["full"]
            comp["rel_change_pct"] = 100.0 * comp["delta"] / comp["baseline"].replace(0, np.nan)
            comp.to_csv(TABLES_DIR / f"summary_median_rrmse_with_delta__{STUDENT_EMB}.csv", index=False)

            log.info(f"summary_median_rrmse__{STUDENT_EMB}.csv + _with_delta__{STUDENT_EMB}.csv written (per-embedding).")


    # ── Per-embedding stats: write only when per-embed outputs are desired, and suffix filenames
    if WRITE_PER_EMBED_TABLES:
        if wil_rows:
            df_w = pd.DataFrame(wil_rows)
            holm_rows = []
            for (regressor, method), grp in df_w.groupby(["regressor","method"]):
                pvals = grp["p_raw"].to_numpy()
                _, p_holm, _, _ = multipletests(pvals, method="holm")
                for i, (_, row) in enumerate(grp.reset_index(drop=True).iterrows()):
                    holm_rows.append({
                        "student":"S1","regressor": regressor,"method": method,
                        "pct": int(row["pct"]), "K": int(row["K"]), "M_max": int(row["M_max"]),
                        "p_raw": float(row["p_raw"]), "p_holm": float(p_holm[i]),
                    })
            pd.DataFrame(holm_rows).sort_values(["regressor","method","pct"]).to_csv(
                TABLES_DIR / f"wilcoxon_holm_vs_baseline__{STUDENT_EMB}.csv", index=False
            )
            log.info(f"wilcoxon_holm_vs_baseline__{STUDENT_EMB}.csv written (per-embedding).")

        if cliffs_rows:
            pd.DataFrame(cliffs_rows).sort_values(["regressor","method","pct"]).to_csv(
                TABLES_DIR / f"cliffs_delta_vs_baseline__{STUDENT_EMB}.csv", index=False
            )
            log.info(f"cliffs_delta_vs_baseline__{STUDENT_EMB}.csv written (per-embedding).")

        # Pooled tests across (method, pct) — per-embedding files
        pooled_wil, pooled_cliffs = [], []
        for (meth, pct), bins in pooled_pairs.items():
            xb = np.asarray(bins["base"], dtype=np.float32)
            xf = np.asarray(bins["full"], dtype=np.float32)
            if xb.size and xb.shape == xf.shape:
                try:
                    _, p_raw = wilcoxon(xf, xb, zero_method="pratt", alternative="less")
                except Exception:
                    p_raw = 1.0
                d, npos, nneg, nzero, mag = paired_cliffs_delta(xf, xb)
                pooled_wil.append({"method": meth, "pct": pct, "n_pairs": int(xb.size), "p_value": float(p_raw)})
                pooled_cliffs.append({"method": meth, "pct": pct, "n_pairs": int(xb.size),
                                      "cliffs_delta_paired": float(d), "npos": int(npos),
                                      "nneg": int(nneg), "nzero": int(nzero), "magnitude": mag})
        if pooled_wil:
            pd.DataFrame(pooled_wil).sort_values(["method","pct"]).to_csv(
                TABLES_DIR / f"wilcoxon_pooled_vs_baseline__{STUDENT_EMB}.csv", index=False
            )
            pd.DataFrame(pooled_cliffs).sort_values(["method","pct"]).to_csv(
                TABLES_DIR / f"cliffs_delta_pooled_vs_baseline__{STUDENT_EMB}.csv", index=False
            )
            log.info(f"pooled tests written (per-embedding) for {STUDENT_EMB}.")

    # ---------------------------------- PLOTS ------------------------------------
    def _color_map_for_pairs(pairs):
        colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
        cyc = cycle(colors)
        mapping = {}
        for pr in pairs:
            mapping[pr] = next(cyc)
        return mapping

    def plot_rrmse_vs_pct(summary_csv_path, pcts=None, emb_only=None):
        df = pd.read_csv(summary_csv_path)
        need = {"embedding","regressor","method","pct","baseline","full"}
        if not need.issubset(df.columns):
            return
        if emb_only:
            df = df[df["embedding"] == emb_only].copy()
        if pcts:
            df = df[df["pct"].isin(list(pcts))].copy()

        for method in sorted(df["method"].unique()):
            sub = df[df["method"] == method].copy()
            agg = sub.groupby(["embedding","regressor","pct"], as_index=False)[["baseline","full"]].median()
            pairs = sorted(agg[["embedding","regressor"]].drop_duplicates().itertuples(index=False, name=None))
            cmap = _color_map_for_pairs(pairs)

            fig, ax = plt.subplots(figsize=(16, 4.8))
            xticks = sorted(set(PCTS_FOR_PLOTS) | set(agg["pct"].unique()))
            ax.set_xticks(xticks)
            ax.set_xlim(min(PCTS_FOR_PLOTS) - 5, max(PCTS_FOR_PLOTS) + 20)

            for emb, reg in pairs:
                cur = agg[(agg["embedding"] == emb) & (agg["regressor"] == reg)].sort_values("pct")
                color = cmap[(emb, reg)]
                ax.plot(cur["pct"], cur["full"],     marker="o", linestyle="-",  color=color, label=f"{emb} | {reg} (full)")
                ax.plot(cur["pct"], cur["baseline"], marker="o", linestyle="--", color=color, label=f"{emb} | {reg} (baseline)")

            ax.set_xlabel("%K")
            ax.set_ylabel("RRMSE")
            title_suffix = f", {emb_only}" if emb_only else ""
            ax.set_title(f"(Global median{title_suffix}) RRMSE vs %K — {method}")
            ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
            fig.tight_layout(rect=[0, 0, 0.80, 1])
            fig.savefig(OUT_PLOTS / f"combined_rrmse_vs_pct__{method}__{emb_only or 'all'}.png", dpi=200, bbox_inches="tight")
            plt.close(fig)

    def plot_delta_vs_pct(summary_csv_path, pcts=None, emb_only=None):
        df = pd.read_csv(summary_csv_path)
        need = {"embedding","regressor","method","pct","baseline","full"}
        if not need.issubset(df.columns):
            return
        if emb_only:
            df = df[df["embedding"] == emb_only].copy()
        if pcts:
            df = df[df["pct"].isin(list(pcts))].copy()
        df["delta"] = df["baseline"] - df["full"]

        for method in sorted(df["method"].unique()):
            sub = df[df["method"] == method].copy()
            agg = sub.groupby(["embedding","regressor","pct"], as_index=False)["delta"].median()
            pairs = sorted(agg[["embedding","regressor"]].drop_duplicates().itertuples(index=False, name=None))
            cmap = _color_map_for_pairs(pairs)

            fig, ax = plt.subplots(figsize=(16, 4.8))
            xticks = sorted(set(PCTS_FOR_PLOTS) | set(agg["pct"].unique()))
            ax.set_xticks(xticks)
            ax.set_xlim(min(PCTS_FOR_PLOTS) - 5, max(PCTS_FOR_PLOTS) + 20)

            for emb, reg in pairs:
                cur = agg[(agg["embedding"] == emb) & (agg["regressor"] == reg)].sort_values("pct")
                color = cmap[(emb, reg)]
                ax.plot(cur["pct"], cur["delta"], marker="o", linestyle="-", color=color, label=f"{emb} | {reg}")

            ax.axhline(0.0, linestyle="--")
            ax.set_xlabel("%K")
            ax.set_ylabel("ΔRRMSE (baseline - full)")
            title_suffix = f", {emb_only}" if emb_only else ""
            ax.set_title(f"(Global median{title_suffix}) ΔRRMSE vs %K — {method}")
            ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
            fig.tight_layout(rect=[0, 0, 0.80, 1])
            fig.savefig(OUT_PLOTS / f"combined_delta_vs_pct__{method}__{emb_only or 'all'}.png", dpi=200, bbox_inches="tight")
            plt.close(fig)

    # Render per-embedding plots from the per-embedding summary (if written)
    sum_path = OUT_TABLES / f"summary_median_rrmse__{STUDENT_EMB}.csv"
    if WRITE_PER_EMBED_PLOTS and WRITE_PER_EMBED_TABLES and sum_path.exists():
        plot_rrmse_vs_pct(sum_path, pcts=PCTS_FOR_PLOTS, emb_only=STUDENT_EMB)
        plot_delta_vs_pct(sum_path, pcts=PCTS_FOR_PLOTS, emb_only=STUDENT_EMB)


    print(f"Done.\nTables → {OUT_TABLES}\nPlots  → {OUT_PLOTS}")


def run_pipeline_compute():
    DO_REFITS = os.getenv("DO_REFITS", "1").lower() in {"1","true","yes"}
    write_run_config()
    if DO_REFITS:
        for emb in STUDENT_EMBEDDINGS:
            run_for_embedding(emb)
    build_combined_reports_across_embeddings()

def run_pipeline_review():
    """
    Artifact-only: do not encode, fit, or refit anything.
    Rebuild combined tables/plots from the per-fold CSVs already on disk.
    """
    write_run_config()
    build_combined_reports_across_embeddings()

# ────────────────────────────────────────────────────────────────────────────
#  Entry-point
# ────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    try:
        if REVIEW_MODE:
            run_pipeline_review()
        else:
            run_pipeline_compute()
    except Exception as e:
        logging.getLogger(__name__).exception("Fatal error in e_3_student_scoring: %s", e)
        print(f"Error: {e}")


/opt/anaconda3/envs/kr8cht_embedding_comparison/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/kr8cht_embedding_comparison/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
2025-10-16 00:35:34 INFO: run_config.json written.
[combined] summary_median_rrmse.csv written (unified, all pcts).
[combined] summary_median_rrmse_with_delta.csv written (unified).
[combined] wilcoxon_holm_vs_baseline.csv written.
[combined] cliffs_delta_vs_baseline.csv written.
[combined] pooled tests written.
[combined] bootstrap_delta_ci.csv written.
